# Step1: Install Librarys

In [1]:
!pip install datasets
!pip install scikit-learn
!pip install numpy gensim scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


# Step2: Import Librarys

In [2]:
from tensorflow.keras.datasets import imdb
from datasets import load_dataset

import numpy as np
import pandas as pd

from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.utils import to_categorical
from keras import layers
from keras.optimizers import RMSprop
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense

# Step3: Set Global Variables

In [3]:
datasetLimit = 10000
datasetLimitTest = int(datasetLimit*0.2)

# Step4: Loading DataSets

In [4]:
(train_data, train_label), (test_data, test_label) = imdb.load_data(num_words=25000)
train_data = list(train_data[:datasetLimit])
train_label = list(train_label[:datasetLimit])
test_data = list(test_data[:datasetLimitTest])
test_label = list(test_label[:datasetLimitTest])

17464789/17464789 [==============================] - 0s 0us/step


# Step5: Add Model

In [5]:
def get_CNN_model(train_data, test_data, train_labels, test_labels, max_pooling, CNN_size, CNN_window, Epochs, max_words=25000, embedding_dim=128, input_length=100, num_clusters=2):
    train_padded = pad_sequences(train_data, maxlen=input_length)
    test_padded = pad_sequences(test_data, maxlen=input_length)

    train_labels_cat = to_categorical(train_labels, num_clusters)
    test_labels_cat = to_categorical(test_labels, num_clusters)

    model = Sequential()
    model.add(Embedding(max_words, embedding_dim, input_length=input_length))
    model.add(Conv1D(CNN_size, CNN_window, activation='relu'))
    if max_pooling:
        model.add(MaxPooling1D(5))
    model.add(Conv1D(CNN_size, CNN_window, activation='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(num_clusters, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit(train_padded, train_labels_cat, epochs=Epochs, batch_size=128, validation_split=0.2)

    test_predictions = model.predict(test_padded)
    predicted_labels = test_predictions.argmax(axis=1)

    return predicted_labels

In [6]:
predictionsCNN = get_CNN_model(train_data, test_data, train_label, test_label, True, 32, 5, 50)

Epoch 1/50
63/63 [==============================] - 16s 194ms/step - loss: 0.6872 - accuracy: 0.5484 - val_loss: 0.6655 - val_accuracy: 0.6875
Epoch 2/50
63/63 [==============================] - 8s 130ms/step - loss: 0.4952 - accuracy: 0.8240 - val_loss: 0.4041 - val_accuracy: 0.8075
Epoch 3/50
63/63 [==============================] - 7s 116ms/step - loss: 0.1973 - accuracy: 0.9289 - val_loss: 0.4674 - val_accuracy: 0.8060
Epoch 4/50
63/63 [==============================] - 6s 95ms/step - loss: 0.0530 - accuracy: 0.9884 - val_loss: 0.5196 - val_accuracy: 0.8030
Epoch 5/50
63/63 [==============================] - 7s 112ms/step - loss: 0.0130 - accuracy: 0.9985 - val_loss: 0.5819 - val_accuracy: 0.8100
Epoch 6/50
63/63 [==============================] - 6s 95ms/step - loss: 0.0046 - accuracy: 0.9999 - val_loss: 0.6316 - val_accuracy: 0.8125
Epoch 7/50
63/63 [==============================] - 7s 115ms/step - loss: 0.0027 - accuracy: 0.9999 - val_loss: 0.6653 - val_accuracy: 0.8085
Epoch 8

# Step6: Create Confusion Matrix

In [7]:
confusion_matrix_CNN = pd.DataFrame({
    'Data': test_data,
    'Label': test_label
})
confusion_matrix_CNN

,Data,Label
0,"[1, 591, 202, 14, 31, 6, 717, 10, 10, 18142, 1...",0
1,"[1, 14, 22, 3443, 6, 176, 7, 5063, 88, 12, 267...",1
2,"[1, 111, 748, 4368, 1133, 2, 24563, 4, 87, 155...",1
3,"[1, 13, 1228, 119, 14, 552, 7, 20, 190, 14, 58...",0
4,"[1, 40, 49, 85, 84, 1040, 146, 6, 783, 254, 43...",1
...,...,...
1995,"[1, 6, 52, 21, 24, 87, 22, 4, 87, 8434, 9, 125...",1
1996,"[1, 14, 22, 9, 691, 6, 338, 193, 23, 4, 154, 2...",0
1997,"[1, 14, 9, 527, 22, 12, 9, 501, 995, 86, 4, 64...",0
1998,"[1, 61, 1640, 65, 39, 2, 39, 4, 2, 2, 1532, 16...",1


In [8]:
predicted_labels_CNN = list(predictionsCNN)
confusion_matrix_CNN['Predicted'] = predicted_labels_CNN
confusion_matrix_CNN

,Data,Label,Predicted
0,"[1, 591, 202, 14, 31, 6, 717, 10, 10, 18142, 1...",0,0
1,"[1, 14, 22, 3443, 6, 176, 7, 5063, 88, 12, 267...",1,1
2,"[1, 111, 748, 4368, 1133, 2, 24563, 4, 87, 155...",1,1
3,"[1, 13, 1228, 119, 14, 552, 7, 20, 190, 14, 58...",0,0
4,"[1, 40, 49, 85, 84, 1040, 146, 6, 783, 254, 43...",1,1
...,...,...,...
1995,"[1, 6, 52, 21, 24, 87, 22, 4, 87, 8434, 9, 125...",1,1
1996,"[1, 14, 22, 9, 691, 6, 338, 193, 23, 4, 154, 2...",0,0
1997,"[1, 14, 9, 527, 22, 12, 9, 501, 995, 86, 4, 64...",0,1
1998,"[1, 61, 1640, 65, 39, 2, 39, 4, 2, 2, 1532, 16...",1,1


In [9]:
def compute_confusion_matrix_results(confusion_matrix):
    conditions = [
        (confusion_matrix['Label'] == 1) & (confusion_matrix['Predicted'] == 1),  # True Positive (TP)
        (confusion_matrix['Label'] == 0) & (confusion_matrix['Predicted'] == 0),  # True Negative (TN)
        (confusion_matrix['Label'] == 0) & (confusion_matrix['Predicted'] == 1),  # False Positive (FP)
        (confusion_matrix['Label'] == 1) & (confusion_matrix['Predicted'] == 0)   # False Negative (FN)
    ]

    choices = ['TP', 'TN', 'FP', 'FN']

    confusion_matrix['result'] = np.select(conditions, choices, default='')

    return confusion_matrix

In [10]:
confusion_matrix_CNN = compute_confusion_matrix_results(confusion_matrix_CNN)

# Step7: Calculate Measurments

In [11]:
def calculate_metrics(confusion_matrix):
    TP = confusion_matrix[confusion_matrix['result'] == 'TP'].shape[0]
    FP = confusion_matrix[confusion_matrix['result'] == 'FP'].shape[0]
    TN = confusion_matrix[confusion_matrix['result'] == 'TN'].shape[0]
    FN = confusion_matrix[confusion_matrix['result'] == 'FN'].shape[0]

    precision = TP / (TP + FP) if TP + FP != 0 else 0
    recall = TP / (TP + FN) if TP + FN != 0 else 0
    accuracy = (TP + TN) / (TP + TN + FP + FN) if TP + TN + FP + FN != 0 else 0

    return precision, recall, accuracy

In [12]:
def print_metrics(confusion_matrix):
    precision, recall, accuracy = calculate_metrics(confusion_matrix)
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"Accuracy: {accuracy}")
    print("-----------------------")

In [13]:
print_metrics(confusion_matrix_CNN)

Precision: 0.7688492063492064
Recall: 0.8132214060860441
Accuracy: 0.7945
-----------------------
